In [1]:
import qualified Data.ByteString as B
import qualified Data.ByteString.Char8 as BC
import qualified Data.IntSet as IS
import qualified Data.Map.Lazy as M

import Data.ByteString.Read (double)
import Data.Maybe (fromJust)
import Data.List

In [2]:
distance (x1, y1) (x2, y2) = sqrt $ (x1-x2)^2 + (y1-y2)^2

content' <- map (map (fst . fromJust . double) . BC.words) . tail . BC.lines <$> B.readFile "tsp.txt"
content = map (\[a, b] -> (a, b)) content'

inserts = zipWith M.insert [1..25] content
points = foldr ($) M.empty inserts

distances = M.fromList [((i,j), dist) | i <- [1..25], j <- [1..25], let dist = distance (points M.! i) (points M.! j)]

In [ ]:
removeOne set = let
    setLs = IS.toList set
    in map (\e -> (e, IS.fromList $ setLs \\ [e])) setLs


g = M.fromList [((i, s), result (i, s)) | i <- [1..25], s <- map IS.fromList (init $ subsequences [1..25])]
    where
        result (i, s) 
            | s == IS.empty  = distances M.! (i, 1)
            | IS.size s == 1 = let
                e = head $ IS.toList s
                c = distances M.! (i, e)
                in c + (g M.! (e, IS.empty))
            | otherwise      = minimum $ map (subresult i) $ removeOne s
        subresult i (j, s) = (distances M.! (i, j)) + (g M.! (j, s))

g M.! (1, IS.fromList [2..25])